In [47]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
# FRED API
from fredapi import Fred
fred = Fred(api_key='YOUR-KEY')

In [2]:
# GETTING DATA
mortgage_30 = fred.get_series_first_release('MORTGAGE30US')
mortgage_15 = fred.get_series_first_release('MORTGAGE15US')
treasury_10 = fred.get_series_first_release('WGS10YR')
treasury_2 = fred.get_series_first_release('WGS2YR')
fed = fred.get_series_first_release('FF')
aaa = fred.get_series_first_release('WAAA')
baa = fred.get_series_first_release('WBAA')

In [6]:
# To Pickle for now
import pickle

series_dict = {
    'mortgage_30': mortgage_30,
    'mortgage_15': mortgage_15,
    'treasury_10': treasury_10,
    'treasury_2': treasury_2,
    'fed': fed,
    'aaa': aaa,
    'baa': baa
}

for key, value in series_dict.items():
    with open(f'{key}.pkl', 'wb') as file:
        pickle.dump(value, file)

In [2]:
# read Pickle
mortgage_30 = pd.read_pickle('mortgage_30.pkl')
#mortgage_15 = pd.read_pickle('mortgage_15.pkl')
treasury_10 = pd.read_pickle('treasury_10.pkl')
treasury_2 = pd.read_pickle('treasury_2.pkl')
fed = pd.read_pickle('fed.pkl')
aaa = pd.read_pickle('aaa.pkl')
baa = pd.read_pickle('baa.pkl')

# Standardizing and Merging Datasets

In [8]:
# Setting all dates to the Friday of the week
def get_friday(date):
    days_until_friday = 4 - date.dayofweek
    friday = date + pd.Timedelta(days=days_until_friday)
    return friday

In [14]:
mortgage_30.index = mortgage_30.index.map(get_friday)
treasury_10.index = treasury_10.index.map(get_friday)
treasury_2.index = treasury_2.index.map(get_friday)
fed.index = fed.index.map(get_friday)
aaa.index = aaa.index.map(get_friday)
baa.index = baa.index.map(get_friday)

In [66]:
# Does Mortgage 30Yr contain all the weeks?

start_date = min(mortgage_30.index)
end_date = max(mortgage_30.index)

dates_check = []
for date in range(0,int((end_date-start_date).days/7)+1):
    dates_check.append(start_date + date*timedelta(days=7))

    
print('Missing Weeks: ')    
mortgage_30[~mortgage_30.index.isin(dates_check)]

Missing Weeks: 


Series([], Name: value, dtype: object)

In [74]:
# Keeping only Data where all features are populated
mortgage = mortgage_30[(mortgage_30.index>=max(mortgage_30.index.min(),treasury_10.index.min(),
                                    treasury_2.index.min(),fed.index.min(),aaa.index.min(),baa.index.min()))]

In [88]:
# Merging all features
df = pd.concat([mortgage,treasury_10,treasury_2,fed,aaa,baa], axis = 1)
df.columns = ['M30Y','T10Y','T2Y','FED','AAA','BAA']
df = df[~df['M30Y'].isnull()]
df.dropna(inplace = True)
df.head()

,M30Y,T10Y,T2Y,FED,AAA,BAA
date,,,,,,
1976-06-04,8.78,7.92,7.21,5.54,8.63,9.9
1976-06-11,8.83,7.88,7.06,5.44,8.63,9.91
1976-06-18,8.88,7.84,7.03,5.47,8.62,9.87
1976-06-25,8.9,7.81,7.0,5.48,8.6,9.87
1976-07-02,8.93,7.86,7.01,5.58,8.63,9.89


# Feature Engineering
Yield Spreads, Term Spreads

In [91]:
def spread(a,b):
    return df.copy()[a]-df.copy()[b]

In [95]:
# 10Y - 2Y Spread
df['T10-2'] = spread('T10Y','T2Y')
# 10Y - FED Spread
df['T10-F'] = spread('T10Y','FED')
# AAA - BAA Spread
df['A-B'] = spread('AAA','BAA')
# 30Y - FED Spread
df['T10-2'] = spread('M30Y','FED')
# AAA - 10Y Spread
df['A-T10'] = spread('AAA','T10Y')

In [97]:
df.head()

,M30Y,T10Y,T2Y,FED,AAA,BAA,T10-2,T10-F,A-B,A-T10
date,,,,,,,,,,
1976-06-04,8.78,7.92,7.21,5.54,8.63,9.9,3.24,2.38,-1.27,0.71
1976-06-11,8.83,7.88,7.06,5.44,8.63,9.91,3.39,2.44,-1.28,0.75
1976-06-18,8.88,7.84,7.03,5.47,8.62,9.87,3.41,2.37,-1.25,0.78
1976-06-25,8.9,7.81,7.0,5.48,8.6,9.87,3.42,2.33,-1.27,0.79
1976-07-02,8.93,7.86,7.01,5.58,8.63,9.89,3.35,2.28,-1.26,0.77


In [98]:
# to pickle
df.to_pickle('df.pkl')